In [1]:
# 1
import os
import json
from requests import post, exceptions

api_key = "tnt_Jjo574TV761_f8zfSpe6hOt5z9AJKATRWJd7G4I5oQOm4pfTY9p80oP"
file_path = "Alice_in_Wonderland.pdf"
                                                                
try:
    with open(file_path, "rb") as f:
        print("file found")
        content = f.read()
except FileNotFoundError:
    print(f"Dosya bulunamadı: {file_path}")
    exit()

metadata = {"title": "alice", "environment": "alice"}

data = {
    "metadata": json.dumps(metadata)
}

files = { "file": (file_path, content), }

headers = {
    "Authorization": f"Bearer {api_key}",
    "Accept": "application/json"
}

try:
    response = post("https://api.ragie.ai/documents", data=data, files=files, headers=headers)
    response.raise_for_status()  
    print("File uploaded successfully!")
except exceptions.RequestException as e:
    print(f"Uploading error: {e}")


file found
File uploaded successfully!


In [ ]:
test = """"test (2).ipynb"	id	description	unit	price
0	10.100.1001	Master stonemason	h	45,00
1	10.100.1002	Master floor tiler	h	45,00
2	10.100.1003	Master floor-and-wall tiler	h	45,00
3	10.100.1004	Master ceramic tiler	h	45,00
4	10.100.1005	Master marble tiler	h	45,00
5	10.100.1006	Master marble cutter	h	45,00
6	10.100.1007	Master mosaic tiler	h	45,00
7	10.100.1008	Master joiner	h	45,00
8	10.100.1009	Master carpenter	h	45,00
9	10.100.1010	Master of insulation	h	45,00
10	10.100.1011	Blaster (Blasting expert)	h	45,00
11	10.100.1012	Master plasterer	h	45,00
12	10.100.1013	Master bricklayer	h	45,00
13	10.100.1014	Master paver	h	45,00
14	10.100.1015	Concrete master	h	45,00
15	10.100.1016	Roof tiler	h	45,00
16	10.100.1017	Master builder	h	45,00
17	10.100.1018	Master blacksmith	h	45,00
18	10.100.1019	Master steel fixer	h	45,00
19	10.100.1020	Plasterboard master	h	45,00
20	10.100.1021	Master welder	h	45,00
21	10.100.1022	Master glazer	h	45,00
22	10.100.1023	Master painter	h	45,00
23	10.100.1024	Master whitewasher	h	45,00
24	10.100.1025	Master upholsterer	h	45,00
25	10.100.1026	Master tinsmith	h	45,00
26	10.100.1027	Master turner	h	45,00
27	10.100.1028	Master linoleum layer	h	45,00
28	10.100.1029	Master varnisher	h	45,00
29	10.100.1030	Timberman (does timbering work)	h	45,00
30	10.100.1031	Master coppersmith	h	45,00
31	10.100.1032	Master aluminum worker	h	45,00
32	10.100.1033	Gypsum board master	h	45,00
33	10.100.1034	Gypsum block master	h	45,00
34	10.100.1035	Scaffolding Construction Worker	h	45,00
35	10.100.1036	Panel Roofer	h	45,00
36	10.100.1037	Concrete Pump Operator	h	53,00
37	10.100.1038	Gypsum board master's helper	h	33,50
38	10.100.1039	Master mosaic tiler's helper	h	33,50
39	10.100.1040	Gypsum Block Master's Helper	h	33,50
40	10.100.1041	Master carpenter's helper	h	33,50
41	10.100.1042	Master of insulation's helper	h	33,50
42	10.100.1043	Plasterboard master's helper	h	33,50
43	10.100.1044	Master plasterer's helper	h	33,50
44	10.100.1045	Master bricklayer's helper	h	33,50
45	10.100.1046	Master blacksmith's helper	h	33,50"""

In [2]:
import requests
import json

environment= "alice"
url = f"https://api.ragie.ai/documents"
headers = {"Authorization": f"Bearer {api_key}"}

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()
    id = data['documents'][0]['id']
    print(f"En son yüklenen dosya idsi = {data['documents'][0]['id']}")
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

En son yüklenen dosya idsi = 5ffa9bc0-4728-4ccb-be1e-af688086579f


In [3]:
import requests
import json



url = f"https://api.ragie.ai/documents/{id}"
headers = {"Authorization": f"Bearer {api_key}"}

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  

    data = response.json()
    print(data)
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

{'id': '5ffa9bc0-4728-4ccb-be1e-af688086579f', 'created_at': '2024-07-25T14:48:54.748661Z', 'updated_at': '2024-07-25T14:48:55.390910Z', 'status': 'partitioning', 'name': 'Alice_in_Wonderland.pdf', 'metadata': {'title': 'alice', 'environment': 'alice'}, 'chunk_count': 0, 'external_id': None}


# our test data : material_table.csv

maybe important columns are Element,type,level and material.

Element,Element_ID,Type,Level,Material,Volume,Area,Length
Walls,348721,Generic - 200mm,Level 1,Default Wall,10.378381544244544,52.32838317982891,13500000.000000002
Walls,348799,Generic - 200mm,Level 1,Default Wall,5.924384085743169,29.87427472612334,7900000.000000001
Walls,348832,Generic - 200mm,Level 1,Default Wall,6.160371282705472,31.070456695746568,8399999.999999981
Walls,348867,Generic - 200mm,Level 1,Default Wall,2.519267765605479,12.704645393438359,3400000.0
Walls,348907,Generic - 200mm,Level 1,Default Wall,4.112708582754127,20.724919696232977,5150000.000000019
Walls,348945,Generic - 200mm,Level 1,Default Wall,3.2446830938760205,16.36313093876025,4499999.999999986
Walls,349579,Generic -100 mm,Level 1,Default Wall,1.7028762234596897,17.028762234596837,4499999.999999986
Walls,349652,Generic -100 mm,Level 1,Default Wall,2.95308057876629,29.6120357117212,7650000.000000001
Walls,350195,Generic -100 mm,Level 1,Default Wall,1.3899473195226815,13.909112479548364,3209999.9999999856
Roofs,349491,Generic - 400mm,Level 2,Default Roof,46.98353802786268,117.45884506965665,


In [6]:
def ragie(test_data):   
    filter_data = {"environment": "alice"}

    data = {"query": f"  {test_data}", "filter": filter_data,"rerank": True, "top_k": 1}
    body = json.dumps(data)

    url = "https://api.ragie.ai/retrievals"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    try:
    
        response = requests.post(url, headers=headers, data=body)
        response.raise_for_status()  

        data = response.json()
        #print(data)
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
    text_list = [chunk['text'] for chunk in data['scored_chunks']]
    combined_text = '\n'.join(text_list)
    return combined_text

In [7]:
import pandas as pd
df = pd.read_csv("material_table.csv")
df = df.iloc[0]
test_data = df.to_json(orient="records")

In [42]:
test_data

'[{"Element":"Walls","Element_ID":348721,"Type":"Generic - 200mm","Level":"Level 1","Material":"Default Wall","Volume":10.3783815442,"Area":52.3283831798,"Length":13500000.0000000019},{"Element":"Walls","Element_ID":348799,"Type":"Generic - 200mm","Level":"Level 1","Material":"Default Wall","Volume":5.9243840857,"Area":29.8742747261,"Length":7900000.0000000009},{"Element":"Walls","Element_ID":348832,"Type":"Generic - 200mm","Level":"Level 1","Material":"Default Wall","Volume":6.1603712827,"Area":31.0704566957,"Length":8399999.9999999814},{"Element":"Walls","Element_ID":348867,"Type":"Generic - 200mm","Level":"Level 1","Material":"Default Wall","Volume":2.5192677656,"Area":12.7046453934,"Length":3400000.0},{"Element":"Walls","Element_ID":348907,"Type":"Generic - 200mm","Level":"Level 1","Material":"Default Wall","Volume":4.1127085828,"Area":20.7249196962,"Length":5150000.0000000186},{"Element":"Walls","Element_ID":348945,"Type":"Generic - 200mm","Level":"Level 1","Material":"Default Wal

In [7]:
ragie(test_data="who is alice ")

'After a time she heard a little pattering of feet in the distance, and she hastily dried her eyes to see what was coming. It was the White Rabbit returning, splendidly dressed, with a pair of white kid gloves in one hand and a large fan in the other : he came trotting along in a great hurry, muttering to himself as he came, “ Oh ! the Duchess, the Duchess ! Oh ! won’t she be savage if I ’ve kept her waiting !” Alice felt so desperate that she was ready to ask help\nC\nInternet\n17\nFit Page\n18\nTHE POOL\nof any one ; so, when the Rabbit came near her, she began, in a low, timid voice, “ If you please, sir——” The Rabbit started violently, dropped the white kid gloves and the fan, and skurried away into the darkness as hard as he could go.\nNavigate\nDigital Interface by BookVirtual Corp. U.S. Patent Pending. © 2000 All Rights Reserved.\nFull Screen On/Off\nControl\nClose Book\nOF TEARS.\nAlice took up the fan and gloves, and, as the hall was very hot, she kept fanning herself all the 